In [3]:
import numpy as np
import pandas as pd
import anndata
import scanpy as sc
import matplotlib 
import seaborn as sns

In [ ]:
sc.settings.verbosity = 1             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

In [ ]:
results_file = '../results_newdataset'  # the file that will store the analysis results

In [ ]:
adata1 = sc.read_10x_mtx(
    './Downloads/new Dataset/M1RE105/M1RE105/embryo1/filtered_feature_bc_matrix/',  # the directory with the `.mtx` file
    var_names='gene_symbols',                # use gene symbols for the variable names (variables-axis index)
    cache=True)                              # write a cache file for faster subsequent reading

In [ ]:
adata1

In [ ]:
Embryo2 = sc.read_10x_mtx(
    './Downloads/new Dataset/M1RE105/M1RE105/embryo2/filtered_feature_bc_matrix/',  # the directory with the `.mtx` file
    var_names='gene_symbols',                # use gene symbols for the variable names (variables-axis index)
    cache=True)               

In [ ]:
Embryo2

# embryo1 

In [ ]:
adata1.var_names_make_unique()



In [ ]:
adata2.var_names_make_unique()
sc.pl.highest_expr_genes(adata1, n_top=20, )

In [ ]:

def highest_expr_genes(
    adata,
    n_top = 25,
    show = None,
    save = None,
    ax = None,
    gene_symbols = None,
    log = False,
    **kwds,
):
    """\
    Fraction of counts assigned to each gene over all cells.

    Computes, for each gene, the fraction of counts assigned to that gene within
    a cell. The `n_top` genes with the highest mean fraction over all cells are
    plotted as boxplots.

    This plot is similar to the `scater` package function `plotHighestExprs(type
    = "highest-expression")`, see `here
    <https://bioconductor.org/packages/devel/bioc/vignettes/scater/inst/doc/vignette-qc.html>`__. Quoting
    from there:

        *We expect to see the “usual suspects”, i.e., mitochondrial genes, actin,
        ribosomal protein, MALAT1. A few spike-in transcripts may also be
        present here, though if all of the spike-ins are in the top 50, it
        suggests that too much spike-in RNA was added. A large number of
        pseudo-genes or predicted genes may indicate problems with alignment.*
        -- Davis McCarthy and Aaron Lun

    Parameters
    ----------
    adata
        Annotated data matrix.
    n_top
        Number of top
    {show_save_ax}
    gene_symbols
        Key for field in .var that stores gene symbols if you do not want to use .var_names.
    log
        Plot x-axis in log scale
    **kwds
        Are passed to :func:`~seaborn.boxplot`.

    Returns
    -------
    If `show==False` a :class:`~matplotlib.axes.Axes`.
    """
    import seaborn as sns  # Slow import, only import if called
    from scipy.sparse import issparse

    # compute the percentage of each gene per cell
    norm_dict = sc.pp.normalize_total(adata, target_sum=100, inplace=False)

    # identify the genes with the highest mean
    if issparse(norm_dict['X']):
        mean_percent = norm_dict['X'].mean(axis=0).A1
        top_idx = np.argsort(mean_percent)[::-1][:n_top]
        counts_top_genes = norm_dict['X'][:, top_idx].A
    else:
        mean_percent = norm_dict['X'].mean(axis=0)
        top_idx = np.argsort(mean_percent)[::-1][:n_top]
        counts_top_genes = norm_dict['X'][:, top_idx]
    columns = (
        adata.var_names[top_idx]
        if gene_symbols is None
        else adata.var[gene_symbols][top_idx]
    )
    counts_top_genes = pd.DataFrame(columns
    )

    return counts_top_genes
highest_expr_genes(adata1).transpose()

#  1-QC

In [ ]:

sc.pp.filter_cells(adata1, min_genes=300)
sc.pp.filter_genes(adata1, min_cells=3)
adata1.var['mt'] = adata1.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata1, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

#  2- Plots

In [ ]:
sc.pl.violin(adata1, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.2, multi_panel=True)

In [ ]:
sc.pl.scatter(adata1,  "total_counts","n_genes_by_counts",color="pct_counts_mt" )

In [ ]:
p1 = sns.displot(adata1.obs["n_genes_by_counts"], bins=800, kde=False)

## maybe all those are doublets but if yes the total count should be doubled but its standared distrupited 

# 3-Filtering

In [ ]:
adata1 = adata1[adata1.obs.pct_counts_mt < 25, :]
adata1 = adata1[adata1.obs.n_genes_by_counts < 6500, :]
adata1 = adata1[adata1.obs.n_genes_by_counts > 1000, :]

# 4- Normalization E1

In [ ]:
sc.pp.normalize_total(adata1, target_sum=1e4)

In [ ]:
sc.pp.log1p(adata1)

In [ ]:
sc.pp.highly_variable_genes(adata1, min_mean=0.0125, max_mean=3, min_disp=0.5)

In [ ]:
#justa  view 
adata1.var.highly_variable[adata1.var.dispersions_norm > 2 ] 

In [ ]:
sc.pl.highly_variable_genes(adata1)

In [ ]:
## save raw data before  processing values and further filtering
adata1.raw = adata1
adata1 = adat1a[:, adata1.var.highly_variable]

In [ ]:
sc.pp.regress_out(adata1, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(adata1, max_value=10)

plotting the linear regression  to understand  what does regress out do

In [ ]:
import scanpy as sc
import matplotlib.pyplot as plt
import statsmodels.api as sm
# Extract the total_counts and pct_counts_mt variables from adata
x = adata1.obs['total_counts'].values
y = adata1.obs['pct_counts_mt'].values

# Perform OLS linear regression
X = sm.add_constant(x)
model = sm.OLS(y, X).fit()

# Get the predicted values from the model
y_pred = model.predict(X)

# Plot the data and the regression line
plt.scatter(x, y)
plt.plot(x, y_pred, color='red')

Principal component analysis

In [ ]:
sc.tl.pca(adata1, svd_solver='arpack')
sc.pl.pca_loadings(adata1)
sc.pl.pca_variance_ratio(adata1, log=True)

#  6- Computing the neighborhood graph &  Embedding the neighborhood graph  & genes plotting

In [ ]:
sc.pp.neighbors(adata1, n_neighbors=20, n_pcs=30)


In [ ]:
sc.tl.umap(adata1)

In [ ]:
sc.pl.umap(adata)

sc.tl.paga is a function from the scanpy library that is used to compute a pseudo-temporal ordering of cells using the pseudotime approximation by graph abstraction (PAGA) algorithm. The adata argument is an AnnData object that contains the data for which you want to compute the pseudo-temporal ordering.

The sc.tl.leiden() function is an alternative to the sc.tl.paga() function for clustering single-cell RNA-sequencing (scRNA-seq) data. It uses the "Leiden" algorithm to identify clusters of cells based on their gene expression patterns.

One key difference between PAGA and Leiden is that PAGA produces a graph-based representation of the data, while Leiden produces a partition of the data into discrete clusters. PAGA is better suited for identifying complex relationships between cells, while Leiden is more suitable for identifying distinct clusters of cells.

Another difference between PAGA and Leiden is the way in which they handle noise and dropouts in the data. PAGA is more robust to noise and missing values, as it estimates the probability that two cells are part of the same branch rather than making a hard assignment. Leiden is more sensitive to noise and missing values and may produce less accurate results in the presence of these issues.

I hope this information is helpful! Let me know if you have any additional questions.





In [ ]:
sc.tl.paga(adata1)
sc.pl.paga(adata1, plot=False)  # remove `plot=False` if you want to see the coarse-grained graph
sc.tl.umap(adata1, init_pos='paga')

In [ ]:
sc.tl.umap(adata1)

Umap  genes plot 

In [ ]:
sc.pl.umap(adata, color=['CST3', 'NKG7', 'PPBP'])

# 7 -Clustering the neighborhood graph

In [ ]:
sc.tl.leiden(adata1)

In [ ]:
sc.pl.umap(adata1, color=['leiden', 'Pax3', 'NKG7'])

save

In [ ]:
adata.write(results_file)

In [ ]:
# Plotting all top 3 genes  agains clusters
for i in range(17):
    sc.pl.umap(adata1, color=['leiden', genelst[i][0], genelst[i][1],genelst[i][2]])
    print(f"this is top 2 genes for cluster {i}")

# 3d script

In [ ]:
sc.tl.umap(adata,n_components=3)
colors=plt.cm.tab20(range(0,7))
color_dict = {}

for i ,color in enumerate(colors):
    color_dict[i] = color
color_dict

umap=adata.obsm["X_umap"]
umap
fig=plt.figure(figsize=(10,10))
ax=fig.add_subplot(projection='3d')
ax.scatter(umap[:,0],umap[:,1],umap[:,2], c=adata.obs.leiden.astype("int").map(color_dict))
for i in range (0,360,2):
    fig=plt.figure(figsize=(10,10))
    ax=fig.add_subplot(projection='3d')
    ax.scatter(umap[:,0],umap[:,1],umap[:,2], c=adata.obs.leiden.astype("int").map(color_dict))
    x_center= (umap[:,0].max()+umap[:,0].min())/2
    y_center= (umap[:,1].max()+umap[:,1].min())/2
    z_center= (umap[:,2].max()+umap[:,2].min())/2
    ax.plot([x_center,x_center],[y_center,y_center],[umap[:,2].min() -2, umap[:,2].max() +2], c="k",lw =5)
    ax.plot([x_center,x_center],[umap[:,1].min() -2, umap[:,1].max() +2],[z_center,z_center], c="k",lw =5)
    ax.plot([umap[:,0].min() -2, umap[:,0].max() +2],[y_center,y_center],[z_center,z_center], c="k",lw =5)
    ax.view_init(20,i)
    ax.axis("off")
    
    plt.savefig(f"{i}.png" ,dpi=100 ,facecolor="white")
    plt.show()
from PIL import Image

# Create the filenames for the images
filenames = [f'{i}.png'.format(i) for i in range(0,360,2)]

# Load the images into a list
images = []
for filename in filenames:
    images.append(Image.open(filename))

# Create a new image object to hold the animation
animation = Image.new(mode=images[0].mode, size=images[0].size)

# Save the animation as an animated GIF
animation.save('3d UMAP.gif', save_all=True, append_images=images, duration=100, loop=0)

##  8 - Finding marker genes

In [ ]:
sc.tl.rank_genes_groups(adata1, 'leiden', method='t-test')
sc.pl.rank_genes_groups(adata1, n_genes=20, sharey=False)


In [ ]:
adata1.uns["rank_genes_groups"]["names"]
genelst=[]
for line in range(20):
    genelst.append(adata1.uns["rank_genes_groups"]["names"][line])
    
genelst[0:5]

In [1]:
# automatic annotations  file

#                                           Embryo 2 QC 

In [ ]:
Embryo2.var_names_make_unique()
sc.pl.highest_expr_genes(Embryo2, n_top=20, )

In [ ]:
sc.pp.filter_cells(Embryo2, min_genes=300)
sc.pp.filter_genes(Embryo2, min_cells=3)
#
Embryo2.var['mt'] = Embryo2.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(Embryo2, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

# plots

In [ ]:
sc.pl.violin(Embryo2, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.1, multi_panel=True)

In [ ]:
sc.pl.scatter(Embryo2,  "total_counts","n_genes_by_counts",color="pct_counts_mt" )

In [ ]:
sns.displot(Embryo2.obs["n_genes_by_counts"], bins=800, kde=False)

In [ ]:
Embryo2.obs

first dataset


# Filtering

In [ ]:
Embryo2 = Embryo2[Embryo2.obs.total_counts < 60000, :]

In [ ]:
Embryo2 = Embryo2[Embryo2.obs.pct_counts_mt < 30, :]

# 5-Normalize

In [ ]:
sc.pp.normalize_total(Embryo2, target_sum=1e4)

In [ ]:
sc.pp.log1p(Embryo2)

In [ ]:
sc.pp.highly_variable_genes(Embryo2, min_mean=0.0125, max_mean=3, min_disp=0.5)

In [ ]:
sc.pl.highly_variable_genes(Embryo2)

In [ ]:
##no idea why the first step step
Embryo2.raw = Embryo2
Embryo2 = Embryo2[:, adata1.var.highly_variable]

In [ ]:
sc.pp.regress_out(Embryo2, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(Embryo2, max_value=10)

PCA

In [ ]:
sc.tl.pca(Embryo2, svd_solver='arpack')
sc.pl.pca_loadings(Embryo2)
sc.pl.pca_variance_ratio(Embryo2, log=True)

#  6- Computing the neighborhood graph &  Embedding the neighborhood graph  & genes plotting

In [ ]:
sc.pp.neighbors(Embryo2, n_neighbors=20, n_pcs=30)

In [ ]:
sc.tl.paga(Embryo2)
sc.pl.paga(Embryo2, plot=False)  # remove `plot=False` if you want to see the coarse-grained graph
sc.tl.umap(Embryo2, init_pos='paga')

In [ ]:
sc.tl.umap(Embryo2)

Umap gene plots

In [ ]:
sc.pl.umap(adata, color=['CST3', 'NKG7', 'PPBP'])

# 7 -Clustering the neighborhood graph

In [ ]:
sc.tl.leiden(Embryo2)

In [ ]:
sc.pl.umap(Embryo2, color=['leiden'])

In [ ]:
# Plotting all top 3 genes vs  clustering
for i in range(17):
    sc.pl.umap(Embryo2, color=['leiden', genelst[i][0], genelst[i][1],genelst[i][2]])
    print(f"this is top 2 genes for cluster {i}")

## Finding marker genes

In [ ]:
sc.tl.rank_genes_groups(Embryo2, 'leiden', method='t-test')
sc.pl.rank_genes_groups(Embryo2, n_genes=30, sharey=False)
Embryo2.uns["rank_genes_groups"]["names"]
genelst=[]
for line in range(20):
    genelst.append(adata1.uns["rank_genes_groups"]["names"][line])
    
genelst[0:5]

In [ ]:
## csv from atlas , try do the gene look up here

In [ ]:
atlasdf=pd.read_csv('./Downloads/E10.5DayEmbryo_markers.csv', )

In [ ]:
atlasdf=atlasdf.loc[:,["gene",'alias','p_val',"avg_diff"]]